# To extract frames with bounding boxes

In [ ]:
import json
import pickle

# Load annotations from the .pkl file
annotations_file = "./data_cache/jaad_database.pkl"

with open(annotations_file, 'rb') as f:
    annotations = pickle.load(f)

# Create a dictionary to store the combined data
combined_data = {}

# Iterate through pedestrian annotations
for vid, vid_data in annotations.items():
    for ped_id, ped_data in vid_data['ped_annotations'].items():
        # Extract pedestrian actions
        pedestrian_actions = ped_data['behavior']
        frames = ped_data['frames']
        bounding_boxes = ped_data['bbox']
        
        # Associate pedestrian actions with frames
        for frame, action, bbox in zip(frames, pedestrian_actions, bounding_boxes):
            combined_data.setdefault(frame, []).append({
                'pedestrian_action': action,
                'bounding_box': bbox,
                'video_id': vid,
                'pedestrian_id': ped_id
            })

# Iterate through vehicle annotations
for vid, vid_data in annotations.items():
    if 'vehicle_annotations' in vid_data:
        for frame, action in vid_data['vehicle_annotations'].items():
            combined_data.setdefault(frame, []).append({
                'vehicle_action': action,
                'video_id': vid
            })

# Write the combined data to a JSON file
with open('combined_data.json', 'w') as json_file:
    json.dump(combined_data, json_file)

In [ ]:
import os

# Get the current directory
current_directory = os.getcwd()

# List all files in the current directory
files = os.listdir(current_directory)

# Print the files
for file in files:
    print(file)

# Creating Bounding Boxes


This create bounding boxes on all the pedestrians in the video

In [ ]:
import os
import cv2
import pickle

# Path to annotations .pkl file
annotations_file = "./data_cache/jaad_database.pkl"

# Path to the directory containing images
images_dir = "./images"

# Output directory to store images with bounding boxes
output_dir = "./images_with_boxes(1)"

# Load the annotation data
with open(annotations_file, 'rb') as f:
    annotation_data = pickle.load(f)

# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Process the first 5 videos
num_videos_to_process = 5
videos_processed = 0

for video_id, video_data in annotation_data.items():
    # Check if we've processed 5 videos
    if videos_processed >= num_videos_to_process:
        break
    
    # Get the number of frames in the video
    num_frames = video_data['num_frames']

    # Define the frame stride to process every 30th frame
    frame_stride = 10

    # Define the maximum number of images to process
    max_images = 6

    # Initialize a counter for the number of images processed
    num_images_processed = 0

    # Process the frames for this video
    for frame_num in range(1, num_frames + 1, frame_stride):
        # Check if we have processed 6 images
        if num_images_processed >= max_images:
            break

        # Get the bounding boxes for this frame
        bounding_boxes = []
        for ped_id, ped_data in video_data['ped_annotations'].items():
            if frame_num in ped_data['frames']:
                index = ped_data['frames'].index(frame_num)
                bbox = ped_data['bbox'][index]
                if isinstance(bbox, list) and len(bbox) == 4:
                    bounding_boxes.append(bbox)
                else:
                    print(f"Invalid bounding box format for frame {frame_num}: {bbox}")

        # Load the image if it exists
        image_file = f"{frame_num:05d}.png"
        image_path = os.path.join(images_dir, video_id, image_file)
        if not os.path.exists(image_path):
            print(f"Image file {image_file} does not exist for video {video_id}. Skipping...")
            continue

        # Read the image
        image = cv2.imread(image_path)
        if image is None:
            print(f"Error reading image file {image_file} for video {video_id}. Skipping...")
            continue

        # Draw bounding boxes on the image
        for bbox in bounding_boxes:
            x1, y1, x2, y2 = map(int, bbox)
            cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)

        # Save the image with bounding boxes
        output_path = os.path.join(output_dir, video_id, image_file)
        os.makedirs(os.path.dirname(output_path), exist_ok=True)
        cv2.imwrite(output_path, image)

        # Increment the count of processed images
        num_images_processed += 1

        print("Processed:", output_path)

    print("All images processed for video:", video_id)
    videos_processed += 1

print("Processing completed for the first 5 videos.")

This creates bounding box on only the first pedestrian of the video

In [ ]:
import os
import cv2
import pickle

# Path to annotations .pkl file
annotations_file = "./data_cache/jaad_database.pkl"

# Path to the directory containing images
images_dir = "./images"

# Output directory to store images with bounding boxes
output_dir = "./images_with_boxes(1)"

# Load the annotation data
with open(annotations_file, 'rb') as f:
    annotation_data = pickle.load(f)

# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Process the first 5 videos
num_videos_to_process = 10
videos_processed = 0

for video_id, video_data in annotation_data.items():
    # Check if we've processed 5 videos
    if videos_processed >= num_videos_to_process:
        break
    
    # Get the number of frames in the video
    num_frames = video_data['num_frames']

    # Define the frame stride to process every 30th frame
    frame_stride = 10

    # Define the maximum number of images to process
    max_images = 15

    # Initialize a counter for the number of images processed
    num_images_processed = 0

    # Process the frames for this video
    for frame_num in range(1, num_frames + 1, frame_stride):
        # Check if we have processed 6 images
        if num_images_processed >= max_images:
            break

        # Get the bounding box for the first pedestrian in this frame
        bounding_box = None
        for ped_id, ped_data in video_data['ped_annotations'].items():
            if frame_num in ped_data['frames']:
                index = ped_data['frames'].index(frame_num)
                bbox = ped_data['bbox'][index]
                if isinstance(bbox, list) and len(bbox) == 4:
                    bounding_box = bbox
                    break

        # Load the image if it exists
        image_file = f"{frame_num:05d}.png"
        image_path = os.path.join(images_dir, video_id, image_file)
        if not os.path.exists(image_path):
            print(f"Image file {image_file} does not exist for video {video_id}. Skipping...")
            continue

        # Read the image
        image = cv2.imread(image_path)
        if image is None:
            print(f"Error reading image file {image_file} for video {video_id}. Skipping...")
            continue

        # Draw bounding box on the image if found
        if bounding_box:
            x1, y1, x2, y2 = map(int, bounding_box)
            cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)

        # Save the image with bounding box if found
        output_path = os.path.join(output_dir, video_id, image_file)
        os.makedirs(os.path.dirname(output_path), exist_ok=True)
        cv2.imwrite(output_path, image)

        # Increment the count of processed images
        num_images_processed += 1

        print("Processed:", output_path)

    print("All images processed for video:", video_id)
    videos_processed += 1

print("Processing completed for all videos.")


In [ ]:
import os
import cv2
import pickle

# Path to the .pkl file
annotations_file = "./data_cache/jaad_database.pkl"

# Directory containing images
images_dir = "./images"

# Directory to save images with bounding boxes
output_dir = "./images_with_boxes"

# Load the annotation data
with open(annotations_file, 'rb') as f:
    annotation_data = pickle.load(f)

# Get the first video ID
video_id = next(iter(annotation_data))

# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Process only the first video
video_data = annotation_data[video_id]

# Print out the keys of the video_data dictionary to inspect its structure
print("Keys in video_data:", video_data.keys())


This creates bounding box on all the pedestrians of the video separately, skipped with every 10th frame

In [ ]:
import os
import cv2
import pickle
import csv
import ast

# Path to annotations .pkl file
annotations_file = "./data_cache/jaad_database.pkl"

# Path to the directory containing images
images_dir = "./images"

# Output directory to store images with bounding boxes
output_main_dir = "./images_with_boxes(Pedestrians Focused)"

# Load the annotation data
with open(annotations_file, 'rb') as f:
    annotation_data = pickle.load(f)

# Load pedestrian annotations from the CSV file
pedestrian_annotations = {}
with open('annotations_output.csv', mode='r') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    for row in csv_reader:
        # Convert the 'Bounding Box' field from string representation of list to actual list
        bounding_box = ast.literal_eval(row['Bounding Box'])
        frames = [int(frame.strip()) for frame in row['Frames'][1:-1].split(',')]
        pedestrian_annotations.setdefault(row['Video ID'], []).append({
            'Frames': frames,
            'Bounding Boxes': bounding_box,
            'Pedestrian ID': row['Pedestrian ID']
        })

# Process the first 10 videos
num_videos_to_process = 346
videos_processed = 0

# Specify the maximum number of images per pedestrian
max_images_per_pedestrian = 5

for video_id, video_data in annotation_data.items():
    if videos_processed >= num_videos_to_process:
        break
    
    # Create a directory for the current video
    output_video_dir = os.path.join(output_main_dir, f"{video_id}")
    os.makedirs(output_video_dir, exist_ok=True)

    # Get the number of frames in the video
    num_frames = video_data['num_frames']

    # Process each pedestrian in the video
    for pedestrian_data_list in pedestrian_annotations.get(video_id, []):
        pedestrian_id = pedestrian_data_list['Pedestrian ID']
        pedestrian_frames = pedestrian_data_list['Frames']
        bounding_boxes = pedestrian_data_list['Bounding Boxes']

        # Create a directory for the current pedestrian
        output_pedestrian_dir = os.path.join(output_video_dir, f"Pedestrian_{pedestrian_id}")
        os.makedirs(output_pedestrian_dir, exist_ok=True)

        # Initialize a counter for the number of images processed for the current pedestrian
        images_processed = 0

        # Process the frames for this pedestrian
        for i in range(0, len(pedestrian_frames), 10):
            # Check if the maximum number of images has been reached
            if images_processed >= max_images_per_pedestrian:
                break

            # Get the frame number
            frame_num = pedestrian_frames[i]

            # Check if the current frame is within the bounds of the video
            if frame_num <= num_frames:
                # Load the image for the current frame
                image_file = f"{frame_num:05d}.png"
                image_path = os.path.join(images_dir, video_id, image_file)
                if not os.path.exists(image_path):
                    print(f"Image file {image_file} does not exist for video {video_id}. Skipping...")
                    continue

                # Read the image
                image = cv2.imread(image_path)
                if image is None:
                    print(f"Error reading image file {image_file} for video {video_id}. Skipping...")
                    continue

                # Draw bounding box for the pedestrian in the frame
                x1, y1, x2, y2 = map(int, bounding_boxes[i])
                cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)

                # Save the image with bounding box
                output_filename = f"Pedestrian_{pedestrian_id}_Image_{frame_num}.png"
                output_path = os.path.join(output_pedestrian_dir, output_filename)
                cv2.imwrite(output_path, image)

                print("Processed:", output_path)

                # Increment the counter for images processed
                images_processed += 1

    print("All images processed for video:", video_id)
    videos_processed += 1

print("Processing completed for all videos.")


This code will generate images for each ped for each frame without skipping any interval

In [ ]:
import os
import cv2
import pickle
import csv
import ast

# Path to annotations .pkl file
annotations_file = "./data_cache/jaad_database.pkl"

# Path to the directory containing images
images_dir = "./images"

# Output directory to store images with bounding boxes
output_main_dir = "./images_with_boxes(Pedestrians Focused)"

# Load the annotation data
with open(annotations_file, 'rb') as f:
    annotation_data = pickle.load(f)

# Load pedestrian annotations from the CSV file
pedestrian_annotations = {}
with open('annotations_output.csv', mode='r') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    for row in csv_reader:
        # Convert the 'Bounding Box' field from string representation of list to actual list
        bounding_box = ast.literal_eval(row['Bounding Box'])
        frames = [int(frame.strip()) for frame in row['Frames'][1:-1].split(',')]
        pedestrian_annotations.setdefault(row['Video ID'], []).append({
            'Frames': frames,
            'Bounding Boxes': bounding_box,
            'Pedestrian ID': row['Pedestrian ID']
        })

# Process the first 10 videos
num_videos_to_process = 346  # Set the end number to 346
videos_processed = 162  # Start processing from video number 163

for video_id, video_data in annotation_data.items():
    if videos_processed > num_videos_to_process:  # Adjusted the condition to stop at the end video
        break

    
    # Create a directory for the current video
    output_video_dir = os.path.join(output_main_dir, f"{video_id}")
    os.makedirs(output_video_dir, exist_ok=True)

    # Get the number of frames in the video
    num_frames = video_data['num_frames']

    # Process each pedestrian in the video
    for pedestrian_data_list in pedestrian_annotations.get(video_id, []):
        pedestrian_id = pedestrian_data_list['Pedestrian ID']
        pedestrian_frames = pedestrian_data_list['Frames']
        bounding_boxes = pedestrian_data_list['Bounding Boxes']

        # Create a directory for the current pedestrian
        output_pedestrian_dir = os.path.join(output_video_dir, f"Pedestrian_{pedestrian_id}")
        os.makedirs(output_pedestrian_dir, exist_ok=True)

        # Process the frames for this pedestrian
        for i, frame_num in enumerate(pedestrian_frames):
            # Check if the current frame is within the bounds of the video
            if frame_num <= num_frames:
                # Load the image for the current frame
                image_file = f"{frame_num:05d}.png"
                image_path = os.path.join(images_dir, video_id, image_file)
                if not os.path.exists(image_path):
                    print(f"Image file {image_file} does not exist for video {video_id}. Skipping...")
                    continue

                # Read the image
                image = cv2.imread(image_path)
                if image is None:
                    print(f"Error reading image file {image_file} for video {video_id}. Skipping...")
                    continue

                # Draw bounding box for the pedestrian in the frame
                x1, y1, x2, y2 = map(int, bounding_boxes[i])
                cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)

                # Save the image with bounding box
                output_filename = f"Pedestrian_{pedestrian_id}_Image_{frame_num}.png"
                output_path = os.path.join(output_pedestrian_dir, output_filename)
                cv2.imwrite(output_path, image)

                print("Processed:", output_path)

    print("All images processed for video:", video_id)
    videos_processed += 1

print("Processing completed for all videos.")


In [ ]:
import os
import cv2
import pickle
import csv
import ast

# Path to annotations .pkl file
annotations_file = "./data_cache/jaad_database.pkl"

# Path to the directory containing images
images_dir = "./images"

# Output directory to store images with bounding boxes
output_main_dir = "./images_with_boxes(Pedestrians Focused)"

# Load the annotation data
with open(annotations_file, 'rb') as f:
    annotation_data = pickle.load(f)

# Load pedestrian annotations from the CSV file
pedestrian_annotations = {}
with open('annotations_output.csv', mode='r') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    for row in csv_reader:
        # Convert the 'Bounding Box' field from string representation of list to actual list
        bounding_box = ast.literal_eval(row['Bounding Box'])
        frames = [int(frame.strip()) for frame in row['Frames'][1:-1].split(',')]
        pedestrian_annotations.setdefault(row['Video ID'], []).append({
            'Frames': frames,
            'Bounding Boxes': bounding_box,
            'Pedestrian ID': row['Pedestrian ID']
        })

# Specify the range of videos to process (starting and ending video numbers)
start_video_num = 165  # Start processing from video number 163
end_video_num = 346    # End processing at video number 346

for video_id, video_data in annotation_data.items():
    # Extract the video number from the video_id
    video_num = int(video_id.split('_')[-1])

    # Check if the current video number is within the specified range
    if start_video_num <= video_num <= end_video_num:
        # Create a directory for the current video
        output_video_dir = os.path.join(output_main_dir, f"{video_id}")
        os.makedirs(output_video_dir, exist_ok=True)

        # Get the number of frames in the video
        num_frames = video_data['num_frames']

        # Process each pedestrian in the video
        for pedestrian_data_list in pedestrian_annotations.get(video_id, []):
            pedestrian_id = pedestrian_data_list['Pedestrian ID']
            pedestrian_frames = pedestrian_data_list['Frames']
            bounding_boxes = pedestrian_data_list['Bounding Boxes']

            # Create a directory for the current pedestrian
            output_pedestrian_dir = os.path.join(output_video_dir, f"Pedestrian_{pedestrian_id}")
            os.makedirs(output_pedestrian_dir, exist_ok=True)

            # Process the frames for this pedestrian
            for i, frame_num in enumerate(pedestrian_frames):
                # Check if the current frame is within the bounds of the video
                if frame_num <= num_frames:
                    # Load the image for the current frame
                    image_file = f"{frame_num:05d}.png"
                    image_path = os.path.join(images_dir, video_id, image_file)
                    if not os.path.exists(image_path):
                        print(f"Image file {image_file} does not exist for video {video_id}. Skipping...")
                        continue

                    # Read the image
                    image = cv2.imread(image_path)
                    if image is None:
                        print(f"Error reading image file {image_file} for video {video_id}. Skipping...")
                        continue

                    # Draw bounding box for the pedestrian in the frame
                    x1, y1, x2, y2 = map(int, bounding_boxes[i])
                    cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)

                    # Save the image with bounding box
                    output_filename = f"Pedestrian_{pedestrian_id}_Image_{frame_num}.png"
                    output_path = os.path.join(output_pedestrian_dir, output_filename)
                    cv2.imwrite(output_path, image)

                    print("Processed:", output_path)

        print("All images processed for video:", video_id)

print("Processing completed for all videos.")
